<a href="https://colab.research.google.com/github/yingzibu/drug_design_JAK/blob/main/chemberta_train_cuda.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [2]:
from google.colab import drive
drive.mount('/content/drive/')

Drive already mounted at /content/drive/; to attempt to forcibly remount, call drive.mount("/content/drive/", force_remount=True).


In [12]:
cd /content/drive/MyDrive/chemberta

/content/drive/MyDrive/chemberta


In [8]:
!pip install transformers  --quiet
!pip install pubchempy --quiet
!pip install rdkit --quiet

In [14]:
import torch
import os
from tqdm import tqdm
import math
import pandas as pd
import numpy as np
import torch.optim as optim
import torch.nn as nn
from torch.utils.data import DataLoader, Dataset
from sklearn.model_selection import KFold, train_test_split
from sklearn.metrics import classification_report, confusion_matrix, average_precision_score, roc_auc_score

MPNN   https://arxiv.org/pdf/1704.01212v2.pdf   

chemberta-2 https://github.com/deepchem/deepchem/blob/master/examples/tutorials/Transfer_Learning_With_ChemBERTa_Transformers.ipynb




In [15]:
from help_function import *
from const import *
from chembertforclassification import *
from bert_encoder import *

class jak_dataset(Dataset):
    def __init__(self, dataframe):
        super(jak_dataset, self).__init__()

        self.len = len(dataframe)
        self.dataframe = dataframe

    def __getitem__(self, idx):
        try:
            sml = self.dataframe.Smiles[idx]
        except:
            sml = self.dataframe.SMILES[idx]

        y = 1 if self.dataframe.Activity[idx] == 1 else 0

        return sml, y

    def __len__(self):
        return self.len

def data_load(df, params={'batch_size': 16, 'shuffle': True,
                          'drop_last': False, 'num_workers': 0}):
    reset_df = df.reset_index(drop=True)
    data = jak_dataset(reset_df)
    loader = DataLoader(data, **params)
    return loader

In [43]:
data_path = 'data_0.25uM/'
if torch.cuda.is_available(): device='cuda'
else: device='cpu'

enzymes = ['JAK1', 'JAK2', 'JAK3', 'TYK2']
model_path = 'model_epoch_20/'
create_path(model_path) # if model_path exists, ignore, if not,create it

In [ ]:
for enzyme in enzymes:
    print()
    ind = enzymes.index(enzyme)
    print(enzyme)
    train_df = pd.read_csv(data_path + enzyme + '_train.csv')
    valid_df = pd.read_csv(data_path + enzyme + '_valid.csv')
    # print(train_df['Activity'].value_counts())
    weight_dict = {1: torch.tensor([3.0, 1.0]), 2: torch.tensor([2.0, 1.0]),
                   3: torch.tensor([2.0, 1.0]), 4: torch.tensor([2.0, 1.0])}
    params = {'batch_size': 16, 'shuffle': True,
              'drop_last': False, 'num_workers': 0}

    train_loader = data_load(train_df, params)
    valid_loader = data_load(valid_df, params)
    epoches = 20
    model_name = f'chembert_{enzyme}.pt'
    file_exist = os.path.isfile(model_path + model_name)
    print(f"{model_path+model_name} existence: ", file_exist)

    if device == 'cuda':
        model = chembertforclassification().cuda()
    else: model = chembertforclassification()

    if file_exist: # load model from predefined model_path
        model.load_state_dict(torch.load(model_path+model_name,
                                         map_location=torch.device(device)))
    else: # trained model does not exist, need to train and save
        optimizer = optim.AdamW(params=model.parameters(),
                                lr=1e-5, weight_decay=1e-2)
        loss_function = nn.CrossEntropyLoss(weight=weight_dict[ind+1].cuda())
        model.train()
        for epoch in range(epoches):
            print("EPOCH -- {}".format(epoch))
            total_loss = 0
            for idx, (x, y_true) in tqdm(enumerate(train_loader),
                                         total=len(train_loader)):
                # print(y_true)
                optimizer.zero_grad()
                output = model(list(x))
                loss = loss_function(output, y_true.cuda())
                loss.backward()
                optimizer.step()
                total_loss += loss.item()
        ###SAVE MODEL
        torch.save(model.state_dict(), model_path + model_name)
        print(f'model trained and saved at {model_path + model_name}')
    model.eval()

    accumulate_y_pred = []
    accumulate_y_true = []
    accumulate_y_prob = []
    accumulate_x = []
    for idx, (x, y_true) in tqdm(enumerate(valid_loader),
                                 total=len(valid_loader)):
        output = model(list(x))
        _, y_pred = torch.max(output, 1)
        accumulate_y_pred.extend(y_pred.tolist())
        accumulate_y_true.extend(y_true.tolist())
        accumulate_y_prob.extend(torch.softmax(output, 1)[:, 1].tolist())
        accumulate_x.extend(list(x))
    evaluate(accumulate_y_true, accumulate_y_pred, accumulate_y_prob)



In [38]:
model_name

'chembert_TYK2.pt'

In [35]:
create_path('model/')

model/  is created


In [36]:
file_name = data_path + 'JAK1_valid.csv'
file_exist = os.path.isfile(file_name)

In [37]:
check_file

True

In [ ]:
model.eval()

100%|██████████| 74/74 [00:00<00:00, 77.47it/s]
